In [1]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

print(tf.__version__)


2.8.0


In [2]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
print(tf.__version__)

2.8.0


In [3]:
# def convnet_model_():
#     vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
#     vgg.trainable = False
#     model = tf.keras.Sequential([
#         #VGG16
#         vgg,
#         #GlobalAveragePooling2D
#         tf.keras.layers.GlobalAveragePooling2D(),
#         #Dense 4096 relu
#         tf.keras.layers.Dense(4096, activation='relu'),
#         # #Dropout 0.5
#         # tf.keras.layers.Dropout(0.5),
#         # #Dense 4096 relu
#         # tf.keras.layers.Dense(4096, activation='relu'),
#         #Dropout 0.5
#         tf.keras.layers.Dropout(0.5),
#         #L2 normalization
#         tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
#     ])

#     return model

def convnet_model_():
    URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
    feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224,3))
    feature_extractor.trainable = False
    model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        #L2 normalization
        tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
    ])

    return model

def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(2048)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


In [4]:
deep_rank_model = deep_rank_model()

# for layer in deep_rank_model.layers:
#     print (layer.name, layer.output_shape)

model_path = "./deep_ranking"

deep_rank_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 96)   18528       ['input_1[0][0]']                
                                                                                              

In [5]:
batch_size = 8
batch_size *= 3


In [6]:
_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    loss =  tf.convert_to_tensor(0,dtype=tf.float32)
    g = tf.constant(1.0, shape=[1], dtype=tf.float32)
    for i in range(0,batch_size,3):
        try:
            q_embedding = y_pred[i+0]
            p_embedding =  y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p =  K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = (loss + g + D_q_p - D_q_n )            
        except:
            continue
    loss = loss/(batch_size/3)
    zero = tf.constant(0.0, shape=[1], dtype=tf.float32)
    return tf.maximum(loss,zero)



In [7]:
pred = deep_rank_model.predict((np.random.rand(24,224,224,3),np.random.rand(24,224,224,3),np.random.rand(24,224,224,3)))
print(pred.shape)
_loss_tensor(None,pred).numpy()

(24, 2048)


array([0.9967773], dtype=float32)

In [8]:

gen1 = ImageDataGenerator(rescale=1./255)
gen1 = gen1.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)
gen2 = ImageDataGenerator(rescale=1./255)
gen2 = gen2.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)
gen3 = ImageDataGenerator(rescale=1./255)
gen3 = gen3.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)

class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1, input_gen2, target_gen):
        self.gen1 = input_gen1
        self.gen2 = input_gen2
        self.gen3 = target_gen

        assert len(input_gen1) == len(input_gen2) == len(target_gen)

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
        x1 = self.gen1[i]
        x2 = self.gen2[i]
        y = self.gen3[i]

        return [x1, x2, y], y

    def on_epoch_end(self):
        self.gen1.on_epoch_end()
        self.gen2.on_epoch_end()
        self.gen3.on_epoch_end()

my_gen = JoinedGen(gen1, gen2, gen3)


Found 3000 images belonging to 1 classes.
Found 3000 images belonging to 1 classes.
Found 3000 images belonging to 1 classes.


In [9]:
#deep_rank_model.load_weights('deepranking.h5')
deep_rank_model.compile(loss=_loss_tensor, optimizer='adam')


# train_steps_per_epoch = int((59520)/batch_size)
train_steps_per_epoch = int((3000)/batch_size)
# train_epochs = 25
train_epochs = 1
deep_rank_model.fit_generator(my_gen,
                        steps_per_epoch=train_steps_per_epoch,
                        epochs=train_epochs
                        )




C:\Users\Timothe\AppData\Local\Temp/ipykernel_25408/2938192524.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  deep_rank_model.fit_generator(my_gen,


125/125 [==============================] - 82s 600ms/step - loss: 0.9990


In [10]:
# model_path = "./deepranking.h5"
model_path = "deepranking"
tf.saved_model.save(deep_rank_model, model_path)
# deep_rank_model.save_weights(model_path)

# test set predictions

In [11]:
deep_rank_model.load_weights('deepranking')

In [ ]:
image1 = np.array(Image.open("food_224/00000.jpg"))
image1 = image1[None]
for i in range(100):
    pred = deep_rank_model.predict([image1, image1, image1])[0]
    print(pred.shape)

In [20]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_224'):
    line_id += 1
    img = np.array(Image.open('food_224/' + filename))[None]
    feature =  deep_rank_model.predict([img, img, img])[0]
    features[filename.replace(".jpg","")] = feature
    if line_id % 100 == 0:
        print(line_id)
        break

# save the features dictionary to a file using pickle
with open('features.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100


In [22]:

features = {}
# load the features dictionary from the file
with open('features.pickle', 'rb') as handle:
    features = pickle.load(handle)
print(features['00001'].shape)

(2048,)


In [16]:

pred = []
line_id = 0
with open('test_triplets.txt', 'r') as f:
    # go through the lines
    for line in f:
        line = line.replace("\n", "")


        embedding1 = features[line.split(' ')[0]]
        embedding2 = features[line.split(' ')[1]]
        embedding3 = features[line.split(' ')[2]]

        
        #compare distance between embeddings
        distance1 = np.sqrt(np.sum((embedding1 - embedding2)**2))
        distance2 = np.sqrt(np.sum((embedding1 - embedding3)**2))

        pred.append(1 if distance1 < distance2 else 0)

        line_id += 1
        if line_id % 100 == 0:
            print(line_id)
            break

np.savetxt("submission.txt", pred, delimiter="\n")


100
